<a href="https://colab.research.google.com/github/MaryMurugami/FI_Phase1_Project/blob/main/Newsgroups_Dataset_Lab_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# To load the data set, we will use sklearn.datasets

from sklearn.datasets import fetch_20newsgroups

categories = [
    'comp.windows.x',
    'rec.sport.hockey',
    'rec.sport.hockey',
    'talk.religion.misc',
    'talk.politics.misc'
]

newsgroups_train = fetch_20newsgroups(
    subset="train",
    remove=('headers', 'footers', 'quotes'),
    categories=categories
)

newsgroups_test = fetch_20newsgroups(
    subset="test",
    remove=('headers', 'footers', 'quotes'),
    categories=categories
)

In [3]:
type(newsgroups_train)

sklearn.utils._bunch.Bunch

We need to extract the feature values using .data and .target to extract target values

In [4]:
# to create X_train, X_test, y_train and y_test using newsgroups_tain and newsgroups_test


import pandas as pd
pd.set_option('max_colwidth', 400)
pd.set_option('use_mathjax', False)

# To extract values from Bunch objects
X_train = pd.DataFrame(newsgroups_train.data, columns=["text"])
X_test = pd.DataFrame(newsgroups_test.data, columns=["text"])
y_train = pd.Series(newsgroups_train.target, name="category")
y_test = pd.Series(newsgroups_test.target, name="category")

In [5]:
# to check shape of the variables

import pandas as pd
pd.set_option('max_colwidth', 400)
pd.set_option('use_mathjax', False)

# Extract values from Bunch objects
X_train = pd.DataFrame(newsgroups_train.data, columns=["text"])
X_test = pd.DataFrame(newsgroups_test.data, columns=["text"])
y_train = pd.Series(newsgroups_train.target, name="category")
y_test = pd.Series(newsgroups_test.target, name="category")

In [7]:
# How many categories are represented in the target distribution dataset

train_target_counts = pd.DataFrame(y_train.value_counts())
train_target_counts["label"] = [newsgroups_train.target_names[val] for val in train_target_counts.index]
train_target_counts.columns = ["count", "target name"]
train_target_counts.index.name = "target value"
train_target_counts

,count,target name
target value,,
1,600,rec.sport.hockey
0,593,comp.windows.x
3,465,talk.politics.misc
4,377,talk.religion.misc


In [8]:
# We will sample 5 records to look at how the full text is displayed in each

train_sample = X_train.sample(5, random_state=22)
train_sample["label"] = [y_train[val] for val in train_sample.index]
train_sample.style.set_properties(**{'text-align': 'left'})

,text,label
973,"> Hey guys! > I work on many stations and would like this name and current logname > to be in a title of Xterm when it's open and a machine name only > when it's closed. In other words, I want $HOST and $LOGNAME to appear > as a title of opened XTerm and $HOST when XTerm is closed. > How can I do it? [Apologies if I'm answering something already answered in the FAQ. Our news feed has been losing a lot of articles lately - so I haven't seen the FAQ lately to check.] Two ways: 1) When you start the xterm, you can use command line options: -n ICON_NAME To set the icon label -T WINDOW_TITLE To set the window title -title WINDOW_TITLE Equivalent to -T WINDOW_TITLE 2) You can use escape sequences to change things on the fly: The basic escape sequence is: ESC ] ; BEL where ESC and BEL are the ASCII ESCAPE and BELL characters, is an ASCII decimal digit -- '2', '1', or '0', depending on whether you are trying to set the window or icon title, or both, and is your desired label string. Hence, this command will set the window & icon title: echo ""\033]0;YOUR_TITLE_GOES_HERE\007\c"" To set just the icon title: echo ""\033]1;YOUR_TITLE_GOES_HERE\007\c"" To set just the window title: echo ""\033]2;YOUR_TITLE_GOES_HERE\007\c"" Of course, you may have to fiddle with exact syntax, depending on how the echo command works on your system and what shell you are using (I've shown System-V semantics under Bourne or Korn shells). [Hint for Sun OS users: use /usr/5bin/echo instead of /bin/echo or Csh's built-in echo. Otherwise you'll have to embed literal ESC and BEL characters in the string instead of using convenient octal sequences.] If you want your titlebar updated continously, say to show your current directory, hostname, or somesuch, then you'll have to see if you can coerce your shell into spitting out the appropriate escape sequences when it prompts for commands. Sometimes you can just put the appropriate escape sequence in the prompt string itself, sometimes not ...",0
1333,"Another dodge. Oh well. I'm no match for your amazing repertoire of red herrings and smoke screens. You asked for an apology. I'm not going to apologize for pointing out that your straw-man argument was a straw-man argument. Nor for saying that your list of ""bible contradictions"" shows such low standards of scholarship that it should be an embarrassment to anti-inerrantists, just as Josh McDowell should be an embarrassment to the fundies. Nor for objecting various times to your taking quotes out of context. Nor for pointing out that ""they do it too"" is not an excuse. Nor for calling your red herrings and smoke screens what they are. I'm still not sure why you think I'm a hypocrite. It's true that I haven't responded to any of Robert Weiss' articles, which may be due in part to the fact that I almost never read his articles. But I have responded to both you and Frank DeCenso (a fundie/inerrantist.) Both you and Frank have taken quotes out of context, and I've objected to both of you doing so. I've criticized bad arguments both when they were yours and I agreed with the conclusion (that the Bible is not inerrant), and when they were Frank's and I disagreed with the conclusion. I've criticized both you and Frank for evading questions, and for trying to ""explain me away"" without addressing the objections I raise (you by accusing me of being hypocritical and irrational, Frank by accusing me of being motivated by a desire to attack the Bible.) I don't see that any of this is hypocritical, nor do I apologize for it. I do apologize, however, for having offended you in any other way. Happy now?",4
663,"NHL RESULTS FOR GAMES PLAYED 4/05/93. -------------------------------------------------------------------------------- STANDINGS PATRICK ADAMS NORRIS SMYTHE TM W L T PT TM W L T PT TM W L T PT TM W L T PT xPIT 53 21 6 112 yMON 47 27 6 100 yDET 44 28 9 97 yVAN 42 28 9 93 WAS 40 31 7 87 yBOS 46 26 7 99 yCHI 43 25 11 97 yCAL 40 29 10 90 NJ 38 35 6 82 yQUE 44 25 1

Data Cleaning and EDA using nltk
To clean the text data, we will standardize the case then tokenize.

In [10]:
rec_sport_hockey_sample = train_sample.iloc[0]["text"]
rec_sport_hockey_sample

'\n  > Hey guys!\n  > I work on many stations and would like this name and current logname\n  > to be in a title of Xterm when it\'s open and a machine name only\n  > when it\'s closed. In other words, I want $HOST and $LOGNAME to appear\n  > as a title of opened XTerm and $HOST when XTerm is closed.\n  > How can I do it?\n\n[Apologies if I\'m answering something already answered in the FAQ.\nOur news feed has been losing a lot of articles lately - so I haven\'t\nseen the FAQ lately to check.]\n\n\nTwo ways:\n \n 1) When you start the xterm, you can use command line options:\n         -n ICON_NAME           To set the icon label\n         -T WINDOW_TITLE        To set the window title\n         -title WINDOW_TITLE    Equivalent to -T WINDOW_TITLE\n\n 2) You can use escape sequences to change things on the fly:\n\n    The basic escape sequence is:\n\tESC ] <P> ; <SOME_ARBITARY_TEXT> BEL\n\n    where ESC and BEL are the ASCII ESCAPE and BELL characters,\n    <P> is an ASCII decimal digit

In [11]:
# Standardize by making everything lowercase

rec_sport_hockey_sample.lower()

'\n  > hey guys!\n  > i work on many stations and would like this name and current logname\n  > to be in a title of xterm when it\'s open and a machine name only\n  > when it\'s closed. in other words, i want $host and $logname to appear\n  > as a title of opened xterm and $host when xterm is closed.\n  > how can i do it?\n\n[apologies if i\'m answering something already answered in the faq.\nour news feed has been losing a lot of articles lately - so i haven\'t\nseen the faq lately to check.]\n\n\ntwo ways:\n \n 1) when you start the xterm, you can use command line options:\n         -n icon_name           to set the icon label\n         -t window_title        to set the window title\n         -title window_title    equivalent to -t window_title\n\n 2) you can use escape sequences to change things on the fly:\n\n    the basic escape sequence is:\n\tesc ] <p> ; <some_arbitary_text> bel\n\n    where esc and bel are the ascii escape and bell characters,\n    <p> is an ascii decimal digit

In [12]:
# To standardize the case in the full dataset using .str.lower()


# To transform sample data to lowercase
train_sample["text"] = train_sample["text"].str.lower()
# To display full text
train_sample.style.set_properties(**{'text-align': 'left'})

,text,label
973,"> hey guys! > i work on many stations and would like this name and current logname > to be in a title of xterm when it's open and a machine name only > when it's closed. in other words, i want $host and $logname to appear > as a title of opened xterm and $host when xterm is closed. > how can i do it? [apologies if i'm answering something already answered in the faq. our news feed has been losing a lot of articles lately - so i haven't seen the faq lately to check.] two ways: 1) when you start the xterm, you can use command line options: -n icon_name to set the icon label -t window_title to set the window title -title window_title equivalent to -t window_title 2) you can use escape sequences to change things on the fly: the basic escape sequence is: esc ] ; bel where esc and bel are the ascii escape and bell characters, is an ascii decimal digit -- '2', '1', or '0', depending on whether you are trying to set the window or icon title, or both, and is your desired label string. hence, this command will set the window & icon title: echo ""\033]0;your_title_goes_here\007\c"" to set just the icon title: echo ""\033]1;your_title_goes_here\007\c"" to set just the window title: echo ""\033]2;your_title_goes_here\007\c"" of course, you may have to fiddle with exact syntax, depending on how the echo command works on your system and what shell you are using (i've shown system-v semantics under bourne or korn shells). [hint for sun os users: use /usr/5bin/echo instead of /bin/echo or csh's built-in echo. otherwise you'll have to embed literal esc and bel characters in the string instead of using convenient octal sequences.] if you want your titlebar updated continously, say to show your current directory, hostname, or somesuch, then you'll have to see if you can coerce your shell into spitting out the appropriate escape sequences when it prompts for commands. sometimes you can just put the appropriate escape sequence in the prompt string itself, sometimes not ...",0
1333,"another dodge. oh well. i'm no match for your amazing repertoire of red herrings and smoke screens. you asked for an apology. i'm not going to apologize for pointing out that your straw-man argument was a straw-man argument. nor for saying that your list of ""bible contradictions"" shows such low standards of scholarship that it should be an embarrassment to anti-inerrantists, just as josh mcdowell should be an embarrassment to the fundies. nor for objecting various times to your taking quotes out of context. nor for pointing out that ""they do it too"" is not an excuse. nor for calling your red herrings and smoke screens what they are. i'm still not sure why you think i'm a hypocrite. it's true that i haven't responded to any of robert weiss' articles, which may be due in part to the fact that i almost never read his articles. but i have responded to both you and frank decenso (a fundie/inerrantist.) both you and frank have taken quotes out of context, and i've objected to both of you doing so. i've criticized bad arguments both when they were yours and i agreed with the conclusion (that the bible is not inerrant), and when they were frank's and i disagreed with the conclusion. i've criticized both you and frank for evading questions, and for trying to ""explain me away"" without addressing the objections i raise (you by accusing me of being hypocritical and irrational, frank by accusing me of being motivated by a desire to attack the bible.) i don't see that any of this is hypocritical, nor do i apologize for it. i do apologize, however, for having offended you in any other way. happy now?",4
663,"nhl results for games played 4/05/93. -------------------------------------------------------------------------------- standings patrick adams norris smythe tm w l t pt tm w l t pt tm w l t pt tm w l t pt xpit 53 21 6 112 ymon 47 27 6 100 ydet 44 28 9 97 yvan 42 28 9 93 was 40 31 7 87 ybos 46 26 7 99 ychi 43 25 11 97 ycal 40 29 10 90 nj 38 35 6 82 yque 44 25 1

In [13]:
# Perform the same on X_train

# To transform text in X_train to lowercase
X_train["text"] = X_train["text"].str.lower()


In [14]:
# To check a sample of the text to see if the tex is lowercase

X_train.iloc[100]["text"]

'we have received a number of requests for a reposting of the\ninternational obfuscated c code contest rules and guidelines.  also\nsome people requested that these rules be posted to a wider set of\ngroups.  sorry for the cross posting.\n\nsome technical clarifications were made to the rules and guidelines.\n(see the diff marks at the right hand edge)  the rules and guidelines\nfor this year remain the same, so people who have already or are\nin the process of submitting entries for the 1993 ioccc need not worry \nabout these changes.\n\nchongo <landon curt noll> /\\cc/\\        chongo@toad.com\nlarry bassel                            lab@sun.com\n\n=-=\n\n#!/bin/sh\n# this is part 02 of a multipart archive\n# ============= mkentry.c ==============\necho "x - extracting mkentry.c (text)"\nsed \'s/^x//\' << \'shar_eof\' > mkentry.c &&\nx/* @(#)mkentry.c\t1.25 4/5/93 15:58:08 */\nx/*\nx * copyright (c) landon curt noll & larry bassel, 1993.\nx * all rights reserved.  permission for pers

TOKENIZING 

In [16]:
# Convert each document into a set of tokens instead of a single long string

religion_sample = train_sample.iloc[1]["text"]
religion_sample

'\nanother dodge.  oh well.  i\'m no match for your amazing repertoire\nof red herrings and smoke screens.  \n\nyou asked for an apology.  i\'m not going to apologize for pointing out\nthat your straw-man argument was a straw-man argument.  nor for saying\nthat your list of "bible contradictions" shows such low standards of\nscholarship that it should be an embarrassment to anti-inerrantists,\njust as josh mcdowell should be an embarrassment to the fundies.  nor\nfor objecting various times to your taking quotes out of context.  nor\nfor pointing out that "they do it too" is not an excuse. nor for calling\nyour red herrings and smoke screens what they are.\n\ni\'m still not sure why you think i\'m a hypocrite.  it\'s true that i\nhaven\'t responded to any of robert weiss\' articles, which may be due in\npart to the fact that i almost never read his articles.  but i have\nresponded to both you and frank decenso (a fundie/inerrantist.)  both\nyou and frank have taken quotes out of contex

In [17]:
# Split this into tokens using the .split method

religion_sample.split()[:10]

['another',
 'dodge.',
 'oh',
 'well.',
 "i'm",
 'no',
 'match',
 'for',
 'your',
 'amazing']

We need to remove the punctuation since when it is attached to a word it becomes a problem. To get the same instance of the same token, we will use the default token pattern in scikit-learn. 

In [21]:
# The default token pattern used in scikit-learn vectorizers
# The RegEX will be

import re
pattern = r'(?u)\b\w\w+\b'



In [22]:
# Uining RegexpTokenizer from NLTK to create tokens
# We will transform the religion sample 


from nltk.tokenize import RegexpTokenizer

basic_token_pattern = r"(?u)\b\w\w+\b"

tokenizer = RegexpTokenizer(basic_token_pattern)
tokenizer.tokenize(religion_sample)[:10]

['another',
 'dodge',
 'oh',
 'well',
 'no',
 'match',
 'for',
 'your',
 'amazing',
 'repertoire']

In [23]:
# To tokenize all the values present in the column
# We will use a pandas dataframe .apply then pass tokenizer.tokenizer


# To create new column with tokenized data
train_sample["text_tokenized"] = train_sample["text"].apply(tokenizer.tokenize)
# Then display full text
train_sample.style.set_properties(**{'text-align': 'left'})

,text,label,text_tokenized
973,"> hey guys! > i work on many stations and would like this name and current logname > to be in a title of xterm when it's open and a machine name only > when it's closed. in other words, i want $host and $logname to appear > as a title of opened xterm and $host when xterm is closed. > how can i do it? [apologies if i'm answering something already answered in the faq. our news feed has been losing a lot of articles lately - so i haven't seen the faq lately to check.] two ways: 1) when you start the xterm, you can use command line options: -n icon_name to set the icon label -t window_title to set the window title -title window_title equivalent to -t window_title 2) you can use escape sequences to change things on the fly: the basic escape sequence is: esc ] ; bel where esc and bel are the ascii escape and bell characters, is an ascii decimal digit -- '2', '1', or '0', depending on whether you are trying to set the window or icon title, or both, and is your desired label string. hence, this command will set the window & icon title: echo ""\033]0;your_title_goes_here\007\c"" to set just the icon title: echo ""\033]1;your_title_goes_here\007\c"" to set just the window title: echo ""\033]2;your_title_goes_here\007\c"" of course, you may have to fiddle with exact syntax, depending on how the echo command works on your system and what shell you are using (i've shown system-v semantics under bourne or korn shells). [hint for sun os users: use /usr/5bin/echo instead of /bin/echo or csh's built-in echo. otherwise you'll have to embed literal esc and bel characters in the string instead of using convenient octal sequences.] if you want your titlebar updated continously, say to show your current directory, hostname, or somesuch, then you'll have to see if you can coerce your shell into spitting out the appropriate escape sequences when it prompts for commands. sometimes you can just put the appropriate escape sequence in the prompt string itself, sometimes not ...",0,"['hey', 'guys', 'work', 'on', 'many', 'stations', 'and', 'would', 'like', 'this', 'name', 'and', 'current', 'logname', 'to', 'be', 'in', 'title', 'of', 'xterm', 'when', 'it', 'open', 'and', 'machine', 'name', 'only', 'when', 'it', 'closed', 'in', 'other', 'words', 'want', 'host', 'and', 'logname', 'to', 'appear', 'as', 'title', 'of', 'opened', 'xterm', 'and', 'host', 'when', 'xterm', 'is', 'closed', 'how', 'can', 'do', 'it', 'apologies', 'if', 'answering', 'something', 'already', 'answered', 'in', 'the', 'faq', 'our', 'news', 'feed', 'has', 'been', 'losing', 'lot', 'of', 'articles', 'lately', 'so', 'haven', 'seen', 'the', 'faq', 'lately', 'to', 'check', 'two', 'ways', 'when', 'you', 'start', 'the', 'xterm', 'you', 'can', 'use', 'command', 'line', 'options', 'icon_name', 'to', 'set', 'the', 'icon', 'label', 'window_title', 'to', 'set', 'the', 'window', 'title', 'title', 'window_title', 'equivalent', 'to', 'window_title', 'you', 'can', 'use', 'escape', 'sequences', 'to', 'change', 'things', 'on', 'the', 'fly', 'the', 'basic', 'escape', 'sequence', 'is', 'esc', 'some_arbitary_text', 'bel', 'where', 'esc', 'and', 'bel', 'are', 'the', 'ascii', 'escape', 'and', 'bell', 'characters', 'is', 'an', 'ascii', 'decimal', 'digit', 'or', 'depending', 'on', 'whether', 'you', 'are', 'trying', 'to', 'set', 'the', 'window', 'or', 'icon', 'title', 'or', 'both', 'and', 'some_arbitary_text', 'is', 'your', 'desired', 'label', 'string', 'hence', 'this', 'command', 'will', 'set', 'the', 'window', 'icon', 'title', 'echo', '033', 'your_title_goes_here', '007', 'to', 'set', 'just', 'the', 'icon', 'title', 'echo', '033', 'your_title_goes_here', '007', 'to', 'set', 'just', 'the', 'window', 'title', 'echo', '033', 'your_title_goes_here', '007', 'of', 'course', 'you', 'may', 'have', 'to', 'fiddle', 'with', 'exact', 'syntax', 'depending', 'on', 'how', 'the', 'echo', 'command', 'works', 'on', 'your', 'system', 'and', 'what', 'shell', 'you', 'are', 'using', 've', 'shown', 'system

We have removed all the letters that had single words such as 'a' , 'have' etc. So now we need to perform EDA after data cleaning. To perform the EDA, we will use NLTK tool FreqDist which will allow us to produce an output resembling a dictionary including their frequencies.

In [30]:
print(X_train.columns)


Index(['text'], dtype='object')


In [39]:
# To create a frequency distribution

from nltk import FreqDist

example_freq_dist = FreqDist(X_train.loc[100, column_name][:20])
example_freq_dist




KeyError: ignored

In [34]:
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

def visualize_top_10(freq_dist, title):

    # Extract data for plotting
    top_10 = list(zip(*freq_dist.most_common(10)))
    tokens = top_10[0]
    counts = top_10[1]

    # Set up plot and plot data
    fig, ax = plt.subplots()
    ax.bar(tokens, counts)

    # Customize plot appearance
    ax.set_title(title)
    ax.set_ylabel("Count")
    ax.yaxis.set_major_locator(MaxNLocator(integer=True))
    ax.tick_params(axis="x", rotation=90)
    
visualize_top_10(example_freq_dist, "Top 10 Word Frequency for Example Tokens")

NameError: ignored